# Bài tập thực hành 3
- Cài đặt Differential Evolution (DE) và Cross Entropy Method (CEM)
- Mã lớp: CS410.N11
- Họ tên: Lê Nhật Huy
- MSSV: 20520056

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/BaitapUIT/CS410/Task3

/content/drive/MyDrive/BaitapUIT/CS410/Task3


# Import necessary package

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display

# Initialize required function

## Objective function

In [ ]:
def Sphere(x): 
    return np.sum(x**2)

def Zakharov(x):
    sum1 = 0
    sum2 = 0
    for i in range(len(x)):
        sum1 += x[i]**2
        sum2 += x[i]*(i+1)
    return sum1 + sum2**2 + sum2**4

def Rosenbrock(x):
    sum = 0
    for i in range(len(x)-1):    
        sum += 100*(x[i+1] - x[i]**2)**2 + (x[i]-1)**2
    return sum

def Michalewicz(x):
    m = 10
    sum = 0
    for i in range(len(x)):    
        sum += np.sin(x[i])*(np.sin(x[i]**2*(i+1)/np.pi))**(2*m)
    return -sum

def Ackley(x):
    a = 20
    b = 0.2
    c = 2*np.pi
    return -a * np.exp(-b*np.sqrt(np.average(x**2))) - np.exp(np.average(np.cos(c*x))) + a + np.exp(1)

In [ ]:
class RewardFunction():
    def __init__(self, function, bound, name):
        self.function = function
        self.bound = bound
        self.name = name
    def __call__(self, x):
        return self.function(x)

In [ ]:
list_of_reward_function = []
list_of_reward_function.append(RewardFunction(lambda x: np.sum(x**2), (-5.12, 5.12), 'Sphere'))
list_of_reward_function.append(RewardFunction(Zakharov, (-5, 10), 'Zakharov'))
list_of_reward_function.append(RewardFunction(Rosenbrock, (-5, 10), 'Rosenbrock')) 
list_of_reward_function.append(RewardFunction(Michalewicz, (0, 3.141569), 'Michalewicz'))
list_of_reward_function.append(RewardFunction(Ackley, (-32.768, 32.768), 'Ackley'))

## Evolutionary  Function

In [ ]:
def de(fobj, bounds, F_scale=0.8, cross_prob=0.7, popsize=20, seed_num = 20520056, num_evaluation = 100000):
    np.random.seed(seed_num)
    dimensions = len(bounds)
    
    lower_bound, upper_bound = np.asarray(bounds).T
    diff = np.fabs(lower_bound - upper_bound)
    pop = lower_bound + diff * np.random.rand(popsize, dimensions) # initialize population
    fitness = np.asarray([fobj(ind) for ind in pop])
    current_num_evaluation = popsize
    current_gen = 0
    best_idx = np.argmin(fitness)
    best = pop[best_idx]
    df = pd.DataFrame(columns=["#gen", "xbest", "f(xbest)", "eval"])
    df = df.append(pd.Series([current_gen, best, fitness[best_idx], current_num_evaluation],
                             index = ["#gen", "xbest", "f(xbest)", "eval"]), 
                   ignore_index=True)
    all_pops= [np.copy(pop)]

    while True:
        current_gen +=1
        for j in range(popsize):
            idxs = [idx for idx in range(popsize) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace=False)]
            mutant = np.clip(a + F_scale * (b-c), lower_bound, upper_bound) # create mutant individual

            cross_points = np.random.rand(dimensions) < cross_prob
            if not np.any(cross_points):
                cross_points[np.random.randint(0, dimensions)] = True
            trial = np.where(cross_points, mutant, pop[j])

            f = fobj(trial)
            current_num_evaluation += 1
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < fitness[best_idx]:
                    best_idx = j
                    best = trial

        df = df.append(pd.Series([current_gen, best, fitness[best_idx], current_num_evaluation],
                                 index = ["#gen", "xbest", "f(xbest)", "eval"]), 
                       ignore_index=True)
        all_pops.append(np.copy(pop))
        if current_num_evaluation >= num_evaluation:
           break
    
    fobj_name = fobj.__name__[3:]
    df.to_csv("logfiles/" + "DE." + fobj_name + "." + str(dimensions) + "." + str(popsize) + "." + str(seed_num) + ".csv")
    return df, all_pops

In [ ]:
def cem(fobj, bounds, n_sample = 20, n_elite = 2, epsilon=0.1, sigma_init = 1.0, seed_num = 20520056, num_evaluation = 100000):
    dimensions = len(bounds)
    lower_bound, upper_bound = np.asarray(bounds).T
    diff = np.fabs(lower_bound - upper_bound)
    mu_current = lower_bound + diff * np.random.rand(dimensions)
    sigma_current = sigma_init * np.identity(dimensions)
    weight = 1/n_elite
    current_num_evaluation = 0
    current_gen = 0
    df = pd.DataFrame(columns=["#gen", "xbest", "f(xbest)", "eval"])
    all_pops = []
    
    while True:
        current_gen += 1
        pop = np.random.multivariate_normal(mu_current, sigma_current, n_sample)
        pop = np.clip(pop, lower_bound, upper_bound)
        pop_fitness = [fobj(ind) for ind in pop]
        current_num_evaluation += n_sample

        index_sort = np.argsort(pop_fitness)[:n_elite]
        pop_elite = pop[index_sort]
        best_idx = index_sort[0]
        best = pop[best_idx]
        mu_current = np.average(pop_elite, axis = 0)

        sigma_current = np.matmul(np.transpose(pop_elite - mu_current), pop_elite - mu_current) * weight
        sigma_current += epsilon * np.identity(dimensions)

        df = df.append(pd.Series([current_gen, best, pop_fitness[best_idx], current_num_evaluation],
                                 index = ["#gen", "xbest", "f(xbest)", "eval"]), 
                       ignore_index=True)
        all_pops.append(np.copy(pop))

        if current_num_evaluation >= num_evaluation:
           break

    fobj_name = fobj.__name__[3:]
    df.to_csv("logfiles/" + "CEM." + fobj_name + "." + str(dimensions) + "." + str(n_sample) + "." + str(seed_num) + ".csv")

    return df, all_pops

# Implement for case d = 2

In [ ]:
list_of_seed = [x for x in range(20520056, 20520056+10)]
# seed = 20520056

In [ ]:
for pop_size in [32, 64, 128, 256, 512, 1024]:
    for i in range(len(list_of_reward_function))
        for seed in list_of_seed:
            print(pop_size, list_of_reward_function[i].name, seed)
            de(list_of_reward_function[i].function, [list_of_reward_function[i].bound]*2, num_evaluation=100000, seed_num=seed, popsize=pop_size)[0]
            # cem(list_of_reward_function[i].function, [list_of_reward_function[i].bound]*2, num_evaluation= 100000, seed_num=seed, n_sample=pop_size)[0]

32 Sphere 20520056
32 Sphere 20520057
32 Sphere 20520058
32 Sphere 20520059
32 Sphere 20520060
32 Sphere 20520061
32 Sphere 20520062
32 Sphere 20520063
32 Sphere 20520064
32 Sphere 20520065
32 Zakharov 20520056
32 Zakharov 20520057
32 Zakharov 20520058
32 Zakharov 20520059
32 Zakharov 20520060
32 Zakharov 20520061
32 Zakharov 20520062
32 Zakharov 20520063
32 Zakharov 20520064
32 Zakharov 20520065
32 Rosenbrock 20520056
32 Rosenbrock 20520057
32 Rosenbrock 20520058
32 Rosenbrock 20520059
32 Rosenbrock 20520060
32 Rosenbrock 20520061
32 Rosenbrock 20520062
32 Rosenbrock 20520063
32 Rosenbrock 20520064
32 Rosenbrock 20520065
32 Michalewicz 20520056
32 Michalewicz 20520057
32 Michalewicz 20520058
32 Michalewicz 20520059
32 Michalewicz 20520060
32 Michalewicz 20520061
32 Michalewicz 20520062
32 Michalewicz 20520063
32 Michalewicz 20520064
32 Michalewicz 20520065
32 Ackley 20520056
32 Ackley 20520057
32 Ackley 20520058
32 Ackley 20520059
32 Ackley 20520060
32 Ackley 20520061
32 Ackley 205200

In [ ]:
for pop_size in [32, 64, 128, 256, 512, 1024]:
    for i in range(len(list_of_reward_function)):
        for seed in list_of_seed:
            print(pop_size, list_of_reward_function[i].name, seed)
            # de(list_of_reward_function[i].function, [list_of_reward_function[i].bound]*2, num_evaluation=100000, seed_num=seed, popsize=pop_size)[0]
            cem(list_of_reward_function[i].function, [list_of_reward_function[i].bound]*2, num_evaluation= 100000, seed_num=seed, n_sample=pop_size)[0]

32 Sphere 20520056
32 Sphere 20520057
32 Sphere 20520058
32 Sphere 20520059
32 Sphere 20520060
32 Sphere 20520061
32 Sphere 20520062
32 Sphere 20520063
32 Sphere 20520064
32 Sphere 20520065
32 Zakharov 20520056
32 Zakharov 20520057
32 Zakharov 20520058
32 Zakharov 20520059
32 Zakharov 20520060
32 Zakharov 20520061
32 Zakharov 20520062
32 Zakharov 20520063
32 Zakharov 20520064
32 Zakharov 20520065
32 Rosenbrock 20520056
32 Rosenbrock 20520057
32 Rosenbrock 20520058
32 Rosenbrock 20520059
32 Rosenbrock 20520060
32 Rosenbrock 20520061
32 Rosenbrock 20520062
32 Rosenbrock 20520063
32 Rosenbrock 20520064
32 Rosenbrock 20520065
32 Michalewicz 20520056
32 Michalewicz 20520057
32 Michalewicz 20520058
32 Michalewicz 20520059
32 Michalewicz 20520060
32 Michalewicz 20520061
32 Michalewicz 20520062
32 Michalewicz 20520063
32 Michalewicz 20520064
32 Michalewicz 20520065
32 Ackley 20520056
32 Ackley 20520057
32 Ackley 20520058
32 Ackley 20520059
32 Ackley 20520060
32 Ackley 20520061
32 Ackley 205200

# Implement for case d = 10

In [ ]:
for pop_size in [32, 64, 128, 256, 512, 1024]:
    for i in range(len(list_of_reward_function)):
        for seed in list_of_seed:
            print(pop_size, list_of_reward_function[i].name, seed)
            de(list_of_reward_function[i].function, [list_of_reward_function[i].bound]*10, num_evaluation=100000, seed_num=seed, popsize=pop_size)[0]
            # cem(list_of_reward_function[i].function, [list_of_reward_function[i].bound]*10, num_evaluation= 100000, seed_num=seed, n_sample=pop_size)[0]

32 Sphere 20520056
32 Sphere 20520057
32 Sphere 20520058
32 Sphere 20520059
32 Sphere 20520060
32 Sphere 20520061
32 Sphere 20520062
32 Sphere 20520063
32 Sphere 20520064
32 Sphere 20520065
32 Zakharov 20520056
32 Zakharov 20520057
32 Zakharov 20520058
32 Zakharov 20520059
32 Zakharov 20520060
32 Zakharov 20520061
32 Zakharov 20520062
32 Zakharov 20520063
32 Zakharov 20520064
32 Zakharov 20520065
32 Rosenbrock 20520056
32 Rosenbrock 20520057
32 Rosenbrock 20520058
32 Rosenbrock 20520059
32 Rosenbrock 20520060
32 Rosenbrock 20520061
32 Rosenbrock 20520062
32 Rosenbrock 20520063
32 Rosenbrock 20520064
32 Rosenbrock 20520065
32 Michalewicz 20520056
32 Michalewicz 20520057
32 Michalewicz 20520058
32 Michalewicz 20520059
32 Michalewicz 20520060
32 Michalewicz 20520061
32 Michalewicz 20520062
32 Michalewicz 20520063
32 Michalewicz 20520064
32 Michalewicz 20520065
32 Ackley 20520056
32 Ackley 20520057
32 Ackley 20520058
32 Ackley 20520059
32 Ackley 20520060
32 Ackley 20520061
32 Ackley 205200

In [ ]:
for pop_size in [32, 64, 128, 256, 512, 1024]:
    for i in range(len(list_of_reward_function)):
        for seed in list_of_seed:
            print(pop_size, list_of_reward_function[i].name, seed)
            # de(list_of_reward_function[i].function, [list_of_reward_function[i].bound]*10, num_evaluation=100000, seed_num=seed, popsize=pop_size)[0]
            cem(list_of_reward_function[i].function, [list_of_reward_function[i].bound]*10, num_evaluation= 100000, seed_num=seed, n_sample=pop_size)[0]

32 Sphere 20520056
32 Sphere 20520057
32 Sphere 20520058
32 Sphere 20520059
32 Sphere 20520060
32 Sphere 20520061
32 Sphere 20520062
32 Sphere 20520063
32 Sphere 20520064
32 Sphere 20520065
32 Zakharov 20520056
32 Zakharov 20520057
32 Zakharov 20520058
32 Zakharov 20520059
32 Zakharov 20520060
32 Zakharov 20520061
32 Zakharov 20520062
32 Zakharov 20520063
32 Zakharov 20520064
32 Zakharov 20520065
32 Rosenbrock 20520056
32 Rosenbrock 20520057
32 Rosenbrock 20520058
32 Rosenbrock 20520059
32 Rosenbrock 20520060
32 Rosenbrock 20520061
32 Rosenbrock 20520062
32 Rosenbrock 20520063
32 Rosenbrock 20520064
32 Rosenbrock 20520065
32 Michalewicz 20520056
32 Michalewicz 20520057
32 Michalewicz 20520058
32 Michalewicz 20520059
32 Michalewicz 20520060
32 Michalewicz 20520061
32 Michalewicz 20520062
32 Michalewicz 20520063
32 Michalewicz 20520064
32 Michalewicz 20520065
32 Ackley 20520056
32 Ackley 20520057
32 Ackley 20520058
32 Ackley 20520059
32 Ackley 20520060
32 Ackley 20520061
32 Ackley 205200